# 使用LSTM对THUCNews新闻进行分类

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Dense, LSTM
import jieba

## 读取数据

In [13]:
df = pd.read_table("./cnews/cnews.test.txt", header=None, names=['label', 'text'])
df.head()

,label,text
0,体育,鲍勃库西奖归谁属？ NCAA最强控卫是坎巴还是弗神新浪体育讯如今，本赛季的NCAA进入到了末...
1,体育,麦基砍28+18+5却充满寂寞 纪录之夜他的痛阿联最懂新浪体育讯上天对每个人都是公平的，贾维...
2,体育,黄蜂vs湖人首发：科比冲击七连胜 火箭两旧将登场新浪体育讯北京时间3月28日，NBA常规赛洛...
3,体育,双面谢亚龙作秀终成做作 谁来为低劣行政能力埋单是谁任命了谢亚龙？谁放纵了谢亚龙？谁又该为谢亚...
4,体育,兔年首战山西换帅后有虎胆 张学文用乔丹名言励志今晚客场挑战浙江稠州银行队，是山西汾酒男篮的兔...


## 数据预处理

在IMDB的案例中，数据是已经处理好的，而现在是原始的数据，因此第一步要对数据进行预处理。将文本转换为神经网络可以处理的格式。

### 分词

和英文不同，中文首先要进行分词，这里使用结巴分词，结巴分词以后返回的是一个生成器，因此还需要转换成列表：

In [14]:
text = df.text.apply(jieba.cut).apply(list)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\18907\AppData\Local\Temp\jieba.cache
Loading model cost 0.630 seconds.
Prefix dict has been built successfully.


In [15]:
text[:5]

0    [鲍勃, 库西, 奖归, 谁, 属, ？,  , NCAA, 最强, 控卫, 是, 坎巴, ...
1    [麦基, 砍, 28, +, 18, +, 5, 却, 充满, 寂寞,  , 纪录, 之夜,...
2    [黄蜂, vs, 湖人, 首发, ：, 科比, 冲击, 七, 连胜,  , 火箭, 两旧, ...
3    [双面, 谢亚龙, 作秀, 终成, 做作,  , 谁, 来, 为, 低劣, 行政, 能力, ...
4    [兔年, 首战, 山西, 换帅, 后, 有, 虎胆,  , 张学文, 用, 乔丹, 名言, ...
Name: text, dtype: object

### 去停用词

分词以后可以进一步去掉常用的一些词：

In [16]:
with open('stopwords/baidu_stopwords.txt', encoding='utf8') as f:
    stop_words = [word.strip() for word in f]

def del_stop_words(words, stop_words):
    return [word for word in words if word not in stop_words]

text_step1 = text.apply(del_stop_words, stop_words=stop_words)

In [17]:
text_step1[:5]

0    [鲍勃, 库西, 奖归, 属, ？,  , NCAA, 最强, 控卫, 坎巴, 弗神, 新浪...
1    [麦基, 砍, 28, +, 18, +, 5, 却, 充满, 寂寞,  , 纪录, 之夜,...
2    [黄蜂, 湖人, 首发, ：, 科比, 冲击, 七, 连胜,  , 火箭, 两旧, 登场, ...
3    [双面, 谢亚龙, 作秀, 终成, 做作,  , 低劣, 行政, 能力, 埋单, 任命, 谢...
4    [兔年, 首战, 山西, 换帅, 后, 虎胆,  , 张学文, 乔丹, 名言, 励志, 今晚...
Name: text, dtype: object

### 去标点符号

可以进一步去掉各种标点符号：

In [18]:
def del_punctuation(words):
    return [
        word for word in words if word not in
        ' \xa0[·’!"\#$%&\'()＃！（）*+,-.。/:;<=>?\@，：?￥★、…．＞【】［］《》？“”‘’\[\\]^_`{|}~]+'
    ]

text_step2 = text_step1.apply(del_punctuation)
text_step2[:5]

0    [鲍勃, 库西, 奖归, 属, NCAA, 最强, 控卫, 坎巴, 弗神, 新浪, 体育讯,...
1    [麦基, 砍, 28, 18, 5, 却, 充满, 寂寞, 纪录, 之夜, 痛, 阿联, 最...
2    [黄蜂, 湖人, 首发, 科比, 冲击, 七, 连胜, 火箭, 两旧, 登场, 新浪, 体育...
3    [双面, 谢亚龙, 作秀, 终成, 做作, 低劣, 行政, 能力, 埋单, 任命, 谢亚龙,...
4    [兔年, 首战, 山西, 换帅, 后, 虎胆, 张学文, 乔丹, 名言, 励志, 今晚, 客...
Name: text, dtype: object

最后还要合并成空格分隔的字符串，才能直接传入keras：

In [19]:
text_step3 = text_step2.apply(lambda x: " ".join(x))
text_step3[:5]

0    鲍勃 库西 奖归 属 NCAA 最强 控卫 坎巴 弗神 新浪 体育讯 如今 本赛季 NCAA...
1    麦基 砍 28 18 5 却 充满 寂寞 纪录 之夜 痛 阿联 最 懂 新浪 体育讯 上天 ...
2    黄蜂 湖人 首发 科比 冲击 七 连胜 火箭 两旧 登场 新浪 体育讯 北京 时间 3 月 ...
3    双面 谢亚龙 作秀 终成 做作 低劣 行政 能力 埋单 任命 谢亚龙 放纵 谢亚龙 谢亚龙 ...
4    兔年 首战 山西 换帅 后 虎胆 张学文 乔丹 名言 励志 今晚 客场 挑战 浙江 稠州 银...
Name: text, dtype: object

### 对标签进行编码

查看标签分类：

In [20]:
df.groupby('label').size()  # size和count不同，size包含nan

label
体育    1000
娱乐    1000
家居    1000
房产    1000
教育    1000
时尚    1000
时政    1000
游戏    1000
科技    1000
财经    1000
dtype: int64

首先需要将中文的标签转换为整数表示,因为keras的`to_categorical`函数只能接受数值类型的输入：

In [21]:
le = LabelEncoder()
label = le.fit_transform(df.label)

# label = label.reshape(-1, 1)  # 注意，如果是使用sklearn的OneHotEncoder，则必须要进行这一步

因为是多分类，还需要将其转换为one-hot编码，一共10个类别，每一行对应着keras的10个输出，所以keras最后的dense层的输出为10，keras的`to_categorical`函数可以直接实现。

In [24]:
label = to_categorical(label)
label

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

### 划分测试验证集

In [25]:
seed = 7
test_size = 0.3

X_train, X_test, Y_train, Y_test = train_test_split(text_step3.values,
                                                    label,
                                                    test_size=test_size,
                                                    shuffle=True,
                                                    random_state=seed)

' '.join([i for i in X_train[0].split(' ')[:20]])

'圣象 地板 免胶 安装 各项 性能指标 合格 国际 专利 锁扣 技术 新型 边缘 防潮 技术 圣象 地板 系列产品 免胶 安装'

### Tokenize

此时`X_train`还都是空格分隔的中文字符串，需要使用keras的`Tokenize`将其转换成整数索引。

In [27]:
max_words = 5000

tok = Tokenizer(num_words=max_words)  # 表示后面进行映射的时候，只映射最常出现的5000个词语
tok.fit_on_texts(X_train)  # 这里相当于根据X_train构建一个字典，比如{"圣象":24, "地板":42}，保存在tok.word_index中

现在可以将所有中文转换成整数索引了：

In [28]:
X_train = tok.texts_to_sequences(X_train)  # 此时才是将字符串转换成列表，并且根据前期构筑的字典将词语映射成整数
X_test = tok.texts_to_sequences(X_test) # 测试集也要转换

' '.join([str(i) for i in X_train[0][:20]])

'1787 146 1377 1962 2314 73 209 2992 209 1787 146 1377 1377 146 1377 59 40 792 1377 1176'

有个细节要注意，即Tokenizer转换成的字典，单词对应的索引是从1开始，而不是0，0是用来做填补：

In [34]:
reverse_word_index = {v:k for k, v in tok.word_index.items()}
reverse_word_index.get(0, '??')  # 可见，tok构筑的字典中不包含索引0

'??'

构造`Tokenizer`实例的时候，有个`num_words`参数，意思是选取最常出现的`num_words`个词语进行映射，比如：

In [35]:
len(tok.word_index)

121368

训练集中一共有12万多个词语，只选取最常出现的5000个进行映射，如果句子中出现不在这5000个之内的词语，则会被丢掉，比如第一句的"专利 锁扣"和"边缘 防潮"这几个词语，映射的时候就被丢掉了。

In [36]:
' '.join([reverse_word_index[i] for i in X_train[0][:20]])

'圣象 地板 安装 各项 合格 国际 技术 新型 技术 圣象 地板 安装 安装 地板 安装 情况 下 不再 安装 制造'

注意，此时生成的每个序列的词语数量不一致，可以用`sequence.pad_sequences`用0进行填充，使每一个序列的长度一致，才能作为神经网络的输入：

In [40]:
len(X_train[0]), len(X_train[1])

(98, 201)

In [42]:
max_len = 600

X_train = sequence.pad_sequences(X_train, maxlen=max_len)  # 表示每一行的长度为max_len，如果不够则前面补0
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

X_train[0][-20:]

array([1787,  146,   80, 3459, 1467,  185, 3338, 1467,  569, 1962,  616,
       2314,   33, 1377,  138, 2242,   91,   23,  569,   23])

## 构建模型

首先就要构筑一个嵌入层，把输入的长度为600的向量转换成稀疏数组，相当于从高维转换到低维，对向量进行压缩，用浮点数来表示。然后是LSTM，最后接dense层。

注意，多分类模型要么损失函数是`categorical_crossentropy`，`metrics`是`accuracy`，要么损失函数是`binary_crossentropy`,`metrics`是`categorical_accuracy`，参考：
- [Keras中的多分类损失函数用法categorical_crossentropy](https://blog.csdn.net/liming89/article/details/106854675/)

In [43]:
embedding_vecor_length = 64
model = Sequential()

model.add(Embedding(max_words, embedding_vecor_length, input_length=max_len))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 600, 64)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 333,802
Trainable params: 333,802
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
88/88 [==============================] - 38s 432ms/step - loss: 2.0057 - accuracy: 0.3330 - val_loss: 1.4473 - val_accuracy: 0.5029
Epoch 2/10
88/88 [==============================] - 38s 430ms/step - loss: 0.9710 - accuracy: 0.6927 - val_loss: 0.5937 - val_accuracy: 0.8250
Epoch 3/10
88/88 [==============================] - 38s 431ms/step - loss: 0.4850 - accuracy: 0.8934 - val_loss: 0.4943 - val_accuracy: 0.8743
Epoch 4/10
88/88 [==============================] - 40s 450ms/step - loss: 0.3244 - accuracy: 0.9286 - val_loss: 0.2867 - val_accuracy: 0.9264
Epoch 5/10
88/88 [==============================] - 39s 439ms/step - loss: 0.1430 - accuracy: 0.9752 - val_loss: 0.2451 - val_accuracy: 0.9386
Epoch 6/10
88/88 [==============================] - 38s 434ms/step - loss: 0.0847 - accuracy: 0.9843 - val_loss: 0.2082 - val_accuracy: 0.9500
Epoch 7/10
88/88 [==============================] - 38s 433ms/step - loss: 0.0919 - accuracy: 0.9836 - val_loss: 0.2191 - val_accuracy: 0.9514

结论：
1. 在LSTM和最后的Dense之间增加Dense，模型变得复杂，可以提升准确率。
2. 去掉停用词和标点符号以后，训练集的准确率提升明显，测试集也略有提升。
3. 增加LSTM的输出维度，在前几轮就达到很高的准确度，训练集的准确率都达到较高水平，但是训练速度变慢。验证集的准确率会略有提升。但有趣的是，验证集和测试集的准确率在最后几轮出现波动，不知道是不是步长较大，导致损失函数在局部最小值周围波动的原因。
4. LSTM增加了dropout和recurrent_dropout层以后，训练集的准确率有所下降，但是验证集的准确率略有提升，最终达到了测试以来的最高值95.35%。这也和预期一致，增加dropout层，可以提升模型的泛化能力。
5. 最后一层激活函数使用softmax，而不是sigmoid以后，训练集的准确率比使用sigmoid要提升快很多，不过验证集相差无几。

## 进行评估

In [45]:
model.evaluate(X_test, Y_test)  # verbose=0则不显示下面的进度条

94/94 [==============================] - 3s 35ms/step - loss: 0.2356 - accuracy: 0.9457


[0.2356480360031128, 0.9456666707992554]

还不错，达到了94.6%的准确度，接下来先保存模型。

In [121]:
model.save('model_width_dense_and_dropout.h5')

## 进行预测

`predict`返回的是每一种类别的概率：

In [116]:
r = model.predict(X_test)

r[0]

array([7.6829195e-03, 9.7234678e-01, 3.3071637e-04, 2.1889806e-04,
       6.3669682e-04, 2.6719868e-03, 2.9242516e-02, 3.9586425e-04,
       6.5881260e-08, 8.4951520e-04], dtype=float32)

如果要直接返回所属的类别，可以使用`predict_classes`方法：

In [117]:
r = model.predict_classes(X_test)

r[:10]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([1, 3, 0, 2, 2, 0, 0, 2, 3, 2], dtype=int64)

In [119]:
r = np.argmax(model.predict(X_test), axis=-1)
r[:10]

array([1, 3, 0, 2, 2, 0, 0, 2, 3, 2], dtype=int64)

然后可以使用前面定义的`LabelEncoder`将整数映射回类别：

In [122]:
results = le.inverse_transform(r)
results[:10]

array(['娱乐', '房产', '体育', '家居', '家居', '体育', '体育', '家居', '房产', '家居'],
      dtype=object)